<a href="https://colab.research.google.com/github/Chamodii/SigmaLaw/blob/party_probability/party_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install stanfordcorenlp

In [0]:
import json
from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'/content/drive/Shared drives/SigmaLaw/stanford-corenlp-full-2018-10-05', quiet=False)
props = {'annotators': 'tokenize, ssplit, pos, lemma, parse, ner, coref, kbp, depparse', 'pipelineLanguage': 'en'}


In [0]:
def potential_party(a):
    return a+1


def subject_check(parties, sentences):
    for each in sentences:
      print(each.get('kbp'))
      if len(each.get('kbp')) > 0:
        for i in parties:
          if each.get('kbp')[0].get('subject') == i:
            parties[i] = potential_party(parties[i])

    print("After subject_check: ", parties)
    return parties

def build_complete_ner(word, ner, l):
  if(l[0].get('ner') == ner):
    word = word + " " + build_complete_ner(l[0].get("word"), l[0].get('ner'), l[1:len(l)])
  #print("built" + word)
  return word

# To update parties based on ner
def NER(parties, sentences):
    # props = {'annotators': 'ner', 'pipelineLanguage': 'en'}
    # result = json.loads(nlp.annotate(text, properties=props))
    # r = result['sentences']

    ner = []
    value = []
    for each in sentences:
      q = each.get("tokens")
    #print(q)

      for j in range(0,len(q)):
        if(q[j].get("ner") == "PERSON" or q[j].get("ner") == "ORGANIZATION"):
          #print("test " + q[j].get("word"))
          word = build_complete_ner(q[j].get("word"), q[j].get("ner"), q[j+1 : len(q)])
        # print("final"+word)
          if(j==0):
            ner.append(word)
          if(j!=0 and q[j-1].get("ner") != q[j].get("ner")):
            ner.append(word)
            value.append(q[j].get("ner"))

    delete = []

    for j in parties:
      party_value = False
      for each in ner:
        if each in j:
          party_value = True
      if party_value == False:
        delete.append(j)






    # for j in parties:
    #     if j in ner:
    #         parties[j] = potential_party(parties[j])
    #     else:
    #       delete.append(j)

# Removing parties that are not PERSON or ORGANIZATION
    for key in delete:
      del parties[key]
    

    if(len(parties)>0):
      maximum = max(parties.values())
      if(maximum>0):
        for j in parties:
          parties[j] = parties[j]/maximum

    print("After NER check: ", parties)

    return parties

In [0]:
def subject_coref(parties, sentences):
  for each in sentences:
    #print(each)
    q = each.get("basicDependencies")
    #print(q)
    for j in q:
      ##dependent is sort of like the index, starts from 1 not 0
      if (j.get("dep") == "nsubj"):
        #print(j.get("dependentGloss")+" : "+str(j.get("dependent")))
        for each in corefs.values():
          for i in each:
            if(i.get('startIndex') <= j.get("dependent") and j.get("dependent") <= i.get('endIndex')):
              parties[each[0].get('text')] = potential_party(parties[each[0].get('text')])
  return parties

In [0]:
def remove_non_ascii_1(text):
  return ''.join(i for i in text if ord(i)<128)

In [0]:
f = open("/content/drive/Shared drives/SigmaLaw/data_2/54111.txt", "rb")
text = f.read()
print(text)

b'Donaghy began his career as an NBA referee in September 1994 and continued in that position for thirteen seasons.  Battista agreed to pay Donaghy a fee for each game in which Donaghy correctly picked the winner. \xe2\x80\x82 Donaghy provided the picks to Martino, Martino relayed the information to Battista, and Battista placed the bets. \n'


In [0]:
stringText = text.decode(errors='replace')
print(stringText)
new_text = remove_non_ascii_1(stringText)

Donaghy began his career as an NBA referee in September 1994 and continued in that position for thirteen seasons.  Battista agreed to pay Donaghy a fee for each game in which Donaghy correctly picked the winner.   Donaghy provided the picks to Martino, Martino relayed the information to Battista, and Battista placed the bets. 



In [0]:
print(new_text)

In 1983, Brenda Marsh's two-year-old son, Phillip Buell, died from a severe head injury while in the care of her then-boyfriend, Kenneth Marsh. Charged with Phillip's death, Marsh claimed that Phillip was injured when he fell off the couch and landed on the fireplace hearth. Marsh was convicted of second-degree murder and imprisoned. Almost two decades later, he filed a second habeas petition, which the San Diego County Superior Court granted at the request of the San Diego District Attorney.



In [0]:
p = nlp.annotate(new_text, properties=props)

In [0]:
result = json.loads(p)

sentences = result['sentences']

In [0]:
print(p)

In [0]:
# Coref clusters
corefs = result['corefs']

print("corefs: ", corefs)

# Calculate rough values for party_probability
parties = {}
init_value = 0.0
for each in corefs.values():
  #print(each)
  parties[each[0].get('text')] = init_value

print(parties)
# Update probability if identified as a subject.
#parties = subject_check(parties, sentences)
parties = subject_coref(parties, sentences)

# Update probability with NER.
parties = NER(parties, sentences)

#print(parties)

corefs:  {'17': [{'id': 9, 'text': 'Battista', 'type': 'PROPER', 'number': 'SINGULAR', 'gender': 'MALE', 'animacy': 'ANIMATE', 'startIndex': 1, 'endIndex': 2, 'headIndex': 1, 'sentNum': 2, 'position': [2, 1], 'isRepresentativeMention': True}, {'id': 17, 'text': 'Battista', 'type': 'PROPER', 'number': 'SINGULAR', 'gender': 'MALE', 'animacy': 'ANIMATE', 'startIndex': 16, 'endIndex': 17, 'headIndex': 16, 'sentNum': 3, 'position': [3, 3], 'isRepresentativeMention': False}], '4': [{'id': 0, 'text': 'Donaghy', 'type': 'PROPER', 'number': 'SINGULAR', 'gender': 'MALE', 'animacy': 'ANIMATE', 'startIndex': 1, 'endIndex': 2, 'headIndex': 1, 'sentNum': 1, 'position': [1, 1], 'isRepresentativeMention': True}, {'id': 4, 'text': 'his', 'type': 'PRONOMINAL', 'number': 'SINGULAR', 'gender': 'MALE', 'animacy': 'ANIMATE', 'startIndex': 3, 'endIndex': 4, 'headIndex': 3, 'sentNum': 1, 'position': [1, 5], 'isRepresentativeMention': False}, {'id': 10, 'text': 'Donaghy', 'type': 'PROPER', 'number': 'SINGULAR'

2
